In [ ]:
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go

import plotly.figure_factory as ff
import numpy as np

import os
import warnings

In [ ]:
try:
    import breaching
except ModuleNotFoundError:
    # You only really need this safety net if you want to run these notebooks directly in the examples directory
    # Don't worry about this if you installed the package or moved the notebook to the main directory.
    import os; os.chdir("..")
    import breaching

In [ ]:
WRITE_TO_FILE = True

In [ ]:
def line(error_y_mode=None,  exponential_trendline=False, **kwargs):
    """Extension of `plotly.express.line` to use error bands."""
    ERROR_MODES = {'bar','band','bars','bands',None}
    if error_y_mode not in ERROR_MODES:
        raise ValueError(f"'error_y_mode' must be one of {ERROR_MODES}, received {repr(error_y_mode)}.")
    if error_y_mode in {'bar','bars',None}:
        fig = px.line(**kwargs)
    elif error_y_mode in {'band','bands'}:
        if 'error_y' not in kwargs:
            raise ValueError(f"If you provide argument 'error_y_mode' you must also provide 'error_y'.")
        figure_with_error_bars = px.line(**kwargs)
        fig = px.line(**{arg: val for arg,val in kwargs.items() if arg != 'error_y'})
        for data in figure_with_error_bars.data:
            x = list(data['x'])
            y_upper = list(data['y'] + data['error_y']['array'])
            y_lower = list(data['y'] - data['error_y']['array'] if data['error_y']['arrayminus'] is None else data['y'] - data['error_y']['arrayminus'])
            color = f"rgba({tuple(int(data['line']['color'].lstrip('#')[i:i+2], 16) for i in (0, 2, 4))},.3)".replace('((','(').replace('),',',').replace(' ','')
            fig.add_trace(
                go.Scatter(
                    x = x+x[::-1],
                    y = y_upper+y_lower[::-1],
                    fill = 'toself',
                    fillcolor = color,
                    line = dict(
                        color = 'rgba(255,255,255,0)'
                    ),
                    hoverinfo = "skip",
                    showlegend = False,
                    legendgroup = data['legendgroup'],
                    xaxis = data['xaxis'],
                    yaxis = data['yaxis'],
                )
            )
            if exponential_trendline:
                coeffs = fit_exponential(data['x'], data['y'])
                xeval = np.logspace(np.log10(min(data['x'])), np.log10(max(data['x'])), num=500)
                fig.add_trace(
                    go.Scatter(x=xeval, y=[f(xx, *coeffs) for xx in xeval], mode='lines', 
                        line=dict(width=2, color=color),
                        hoverinfo = "skip",
                        showlegend = False,
                        legendgroup = data['legendgroup'],
                        xaxis = data['xaxis'],
                        yaxis = data['yaxis'])
                )
        # Reorder data as said here: https://stackoverflow.com/a/66854398/8849755
        reordered_data = []
        for i in range(int(len(fig.data)/2)):
            reordered_data.append(fig.data[i+int(len(fig.data)/2)])
            reordered_data.append(fig.data[i])
        fig.data = tuple(reordered_data)
    return fig

In [ ]:
def default_styling(fig):
    
    fig.update_layout(
        font=dict(
            family="Times New Roman",
            size=20,      
            ),
        legend=dict(
            yanchor="top",
            y=0.24,
            xanchor="right",
            x=1,
            bgcolor="white",
            bordercolor="Black",
            borderwidth=1
        )
    )
    
    fig.update_layout(uniformtext_minsize=28, uniformtext_mode='hide',font=dict(family="Times New Roman"))
    

    fig.update_xaxes(showline=True, showgrid=True, gridwidth=0.1, 
                     gridcolor='rgba(1,1,1,0.15)', 
                     linecolor='black',
                    zerolinecolor='rgba(1,1,1,0.25)')
    fig.update_yaxes(showline=True, showgrid=True, gridwidth=0.1, 
                     gridcolor='rgba(1,1,1,0.15)', 
                     linecolor='black',
                     zerolinecolor='rgba(1,1,1,0.25)')
    fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512],
    )
)

    fig.update_layout(
        margin=dict(l=20, r=30, t=20, b=0),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        width=750,
        height=500,
    )    
    
#     fig.show()
#     fig.write_image(f"images/{key}.pdf")

In [ ]:
logfiles = "outputs/"

In [ ]:
os.listdir(logfiles)

In [ ]:
experiments = ["UNDIV", "S1", "S25", "S2", "S3", "SBBF", "TFirst", "tokenreplace", 
              "noEXT_trafo3_baseparams", "baseparams", "noEXT"]

In [ ]:
def simplify_name(name):
    for exp in experiments:
        if exp in name:
            return exp
    return "base"

In [ ]:
def _retrieve_table(output_folder, name):
        file = os.listdir(os.path.join(output_folder, "tables"))[-1]
        file_location = os.path.join(output_folder, "tables", file)
        # print(file_location)

        df = pd.read_csv(file_location, sep='\t')
        df["dataset"] = file_location.split("training_")[1].split("_reports.csv")[0]
        df["seq_length"] = [int(vals) for vals in name.split("_") if vals.isdigit()][0]
        df["experiment"] = df["name"].map(simplify_name)   
        return df

In [ ]:
import glob

In [ ]:
def get_result(folderlist):
    dfs = dict()
    for name in folderlist:
        if "decepticon" in name:
            output_folder = logfiles + name
            candidate_runs = sorted(glob.glob(output_folder + '/*/*'), reverse=True)
            # print(candidate_runs)
            for candidate in candidate_runs:
            # lastdate = sorted(os.listdir(output_folder))[-1]
                if "2022-07-" in candidate or "2022-08-" in candidate:
                    # print(candidate)
                    # output_folder = os.path.join(output_folder, lastdate)
                    # lasttimestamp = sorted(os.listdir(output_folder))[-1]
                    # output_folder = os.path.join(output_folder, candidate)
                    try:
                        df = _retrieve_table(candidate, name)
                        dfs[name] = df
                        break
                    except Exception as e :
                        if "No such file or directory" not in str(e):
                            print(f"skipped {name} with exception {e}")

    return dfs

In [ ]:
dfs = get_result(os.listdir("outputs"))
len(dfs)

In [ ]:
metric = "max-sentence_accuracy" # "max-sentence_accuracy"

In [ ]:
main_table = pd.DataFrame()
summaries = []
for df in dfs.values():
    try:
        summary = (
        df
        .groupby(["name", "datapoints", "seq_length", "experiment", "dataset", "model"], as_index=False)[metric]
        .agg(["mean", "std", "count"])
        .reset_index()
        )
        summaries.append(summary)
    except:
        pass
    
main_table = pd.concat(summaries)
main_table["tokens_recovered"] = main_table["datapoints"] * main_table["seq_length"] * main_table["mean"]
main_table = main_table.sort_values("seq_length")
main_table = main_table.sort_values("datapoints")
# main_table = main_table[~main_table.name.str.contains("gpt_sanity")]

In [ ]:
main_table[(main_table.seq_length==512) & (main_table.dataset=="stackoverflow")]

In [ ]:
seq_length = 32

In [ ]:
reduced_table = main_table[(main_table.dataset=="wikitext") & (main_table.seq_length == seq_length)]

fig = px.scatter(reduced_table, x="datapoints", y="mean", title='Hyperparameter Variations', 
                 color="experiment", symbol="model",
                                 labels={ # replaces default labels by column name
                "experiment": "Experiment",  
                "mean": f"Total Accuracy", 
                "real_size": "Real Dataset Samples",
                "datapoints": "Batch Size",
            })
fig.update_traces(mode='lines+markers', line=dict(width=3, dash='solid'))


# default_styling(fig)
fig.update_xaxes(type="log")
# fig.update_layout(legend=dict(y=0.39, x=1))
fig.show()

# Model Variations

In [ ]:
seq_length = 32

In [ ]:
reduced_table = main_table[(main_table.dataset=="wikitext") 
                           & (main_table.seq_length == seq_length)]

# reduced_table = reduced_table[~reduced_table.name.str.contains("gpt_sanity")]

reduced_table = reduced_table.groupby(["model", "datapoints", "seq_length"], as_index=False).mean().reset_index()
df_t = reduced_table#[reduced_table.experiment.isin(["base"])]

df_t = df_t.replace({
                        'gpt2S':"GPT-2", 
                        'transformer3':"3-layer Transformer", 
                        'bert-sanity-check':"BERT-base",  
                        }
                         )

fig = px.scatter(data_frame=df_t, x="datapoints", y="mean", color="model",  #error_y="std", error_y_mode="band",
                                 labels={ # replaces default labels by column name
                "experiment": "Experiment",  
                "mean": f"Total Accuracy" if metric=="accuracy" else "Accuracy on Most Leaked Sequence", 
                "real_size": "Real Dataset Samples",
                "datapoints": "Batch Size",
                "model": "Model",
            })
fig.update_traces(mode='lines+markers', line=dict(width=5, dash='solid'))


default_styling(fig)

fig.update_xaxes(type="log")
#fig.update_xaxes(range=[2,])
fig.update_layout(legend=dict(y=0.31, x=0.315))
fig.show()

if WRITE_TO_FILE:
    fig.write_image(f"images/decepticon_model_scaling_{metric}_{seq_length}.pdf")
    fig.write_image(f"images/decepticon_model_scaling_{metric}_{seq_length}.pdf")

In [ ]:
seq_length = 512

In [ ]:
reduced_table = main_table[(main_table.dataset=="stackoverflow") 
                           & (main_table.seq_length == seq_length)]

# reduced_table = reduced_table[~reduced_table.name.str.contains("gpt_sanity")]
reduced_table = reduced_table.groupby(["model", "datapoints", "seq_length"], as_index=False).mean().reset_index()


df_t = reduced_table# [reduced_table.experiment.isin(["base"])]

df_t = df_t.replace({
                        'gpt2S':"GPT-2", 
                        'transformer3':"3-layer Transformer", 
                        'bert-sanity-check':"BERT-base",  
                        }
                         )

fig = px.scatter(data_frame=df_t, x="datapoints", y="mean", color="model",  #error_y="std", error_y_mode="band",
                                 labels={ # replaces default labels by column name
                "experiment": "Experiment",  
                "mean": f"Total Accuracy" if metric=="accuracy" else "Accuracy on Most Leaked Sequence", 
                "real_size": "Real Dataset Samples",
                "datapoints": "Batch Size",
                "model": "Model",
            })
fig.update_traces(mode='lines+markers', line=dict(width=5, dash='solid'))


default_styling(fig)

fig.update_xaxes(type="log")
#fig.update_xaxes(range=[2,])
fig.update_layout(legend=dict(y=0.31, x=0.315))
fig.show()

if WRITE_TO_FILE:
    fig.write_image(f"images/decepticon_model_scaling_{metric}_{seq_length}.pdf")
    fig.write_image(f"images/decepticon_model_scaling_{metric}_{seq_length}.pdf")

In [ ]:
fig = line(data_frame=df_t, x="datapoints", y="mean", color="model",  error_y="std", error_y_mode="band",
                                 labels={ # replaces default labels by column name
                "experiment": "Experiment",  
                "mean": f"Total Accuracy" if metric=="accuracy" else "Accuracy on Most Leaked Sequence", 
                "real_size": "Real Dataset Samples",
                "datapoints": "Batch Size",
            })
fig.update_traces(mode='lines+markers', line=dict(width=5, dash='solid'))


default_styling(fig)

fig.update_xaxes(type="log")
#fig.update_xaxes(range=[2,])
fig.update_layout(legend=dict(y=0.39, x=0.257))
fig.show()

# Threshold Sensitivity

In [ ]:
seq_length = 32

In [ ]:
reduced_table = main_table[(main_table.dataset=="wikitext") 
                           & (main_table.seq_length == seq_length) 
                           & (main_table.model=="gpt2S")]
df_t = reduced_table[reduced_table.experiment.isin(["S1", "S25", "S2", "base"])]

df_t = df_t.replace({
                        'S1':"sd=1", 
                        'base':"sd=1.5", 
                        'S2':"sd=2", 
                        'S25':"sd=2.5", 
                        }
                         )

fig = px.scatter(data_frame=df_t, x="datapoints", y="mean", color="experiment",  #
                 #error_y="std", error_y_mode="band",
                                 labels={ # replaces default labels by column name
                "experiment": "Experiment",  
                "mean": f"Total Accuracy" if metric=="accuracy" else "Accuracy on most Leaked Sequence", 
                "real_size": "Real Dataset Samples",
                "datapoints": "Batch Size",
            })
fig.update_traces(mode='lines+markers', line=dict(width=5, dash='solid'))
fig.data[1].line.dash="dash"
fig.data[2].line.dash="dashdot"
fig.data[3].line.dash="dot"

default_styling(fig)

fig.update_xaxes(type="log")
#fig.update_xaxes(range=[2,])
fig.update_layout(legend=dict(y=0.39, x=0.2))
fig.show()

if WRITE_TO_FILE:
    fig.write_image(f"images/decepticon_embedding_threshold_{metric}.pdf")
    fig.write_image(f"images/decepticon_embedding_threshold_{metric}.pdf")

# No External Data

In [ ]:
reduced_table = main_table[(main_table.dataset=="wikitext") 
                           & (main_table.seq_length == seq_length) 
                           & (main_table.model=="gpt2S")]

reduced_table = reduced_table[~reduced_table.name.str.contains("gptS")]

df_ext = reduced_table[reduced_table.experiment.isin(["noEXT", "base"])]

df_ext = df_ext.replace({
                        'base':"Estimated on public Data", 
                        'noEXT':"Estimated on random data", 
                        }
                         )

fig = px.scatter(data_frame=df_ext, x="datapoints", y="mean", color="experiment", 
                 # error_y="std", error_y_mode="band",
                                 labels={ # replaces default labels by column name
                "experiment": "Experiment",  
                "mean": f"Total Accuracy" if metric=="accuracy" else "Accuracy on Most Leaked Sequence",  
                "real_size": "Real Dataset Samples",
                "datapoints": "Batch Size",
            })
fig.update_traces(mode='lines+markers', line=dict(width=5, dash='solid'))
fig.data[1].line.dash="dash"


default_styling(fig)
fig.update_xaxes(type="log")
fig.update_layout(legend=dict(y=0.25, x=0.4))
fig.show()
if WRITE_TO_FILE:
    fig.write_image(f"images/decepticon_ext_vs_simualated_{metric}.pdf")
    fig.write_image(f"images/decepticon_ext_vs_simualated_{metric}.pdf")

# Dataset variations:

In [ ]:
model = "gpt2S"
seq_length = 32

In [ ]:
reduced_table = main_table[(main_table.seq_length == seq_length) 
                           & (main_table.model==model) & (main_table.experiment=="base")]
reduced_table = reduced_table[~reduced_table.name.str.contains("gptS")]

In [ ]:
fig = px.scatter(reduced_table, x="datapoints", y="mean", color="dataset", #symbol="is_base",
                                 labels={ # replaces default labels by column name
                "experiment": "Experiment",  
                "mean": f"Total Accuracy" if metric=="accuracy" else "Accuracy on Most Leaked Sequence", 
                "real_size": "Real Dataset Samples",
                "datapoints": "Batch Size",
                "dataset":"Dataset",
            })
fig.update_traces(mode='lines+markers', line=dict(width=5, dash='solid'))


default_styling(fig)
fig.update_xaxes(type="log")
fig.update_layout(legend=dict(y=0.39, x=0.257))
fig.show()

if WRITE_TO_FILE:
    fig.write_image(f"images/decepticon_data_sources_{metric}.pdf")
    fig.write_image(f"images/decepticon_data_sources_{metric}.pdf")

In [ ]:
fig = line(data_frame=reduced_table, x="datapoints", y="mean", color="dataset", 
           error_y="std", error_y_mode="band", 
                                 labels={ # replaces default labels by column name
                "experiment": "Experiment",  
                "mean": f"Total Accuracy", 
                "real_size": "Real Dataset Samples",
                "datapoints": "Batch Size",
                "dataset":"Dataset",
            })
fig.update_traces(mode='lines+markers', line=dict(width=5, dash='solid'))


default_styling(fig)
fig.update_xaxes(type="log")
fig.update_layout(legend=dict(y=0.39, x=0.257))
fig.show()

# Print recovered total tokens

In [ ]:
model = "transformer3"
seq_length = 32

In [ ]:
reduced_table = main_table[(main_table.seq_length == seq_length) & (main_table.model==model) 
                           & (main_table.experiment=="base")]
# reduced_table = reduced_table[~reduced_table.name.str.contains("gptS")]

In [ ]:
if metric == "accuracy":
    fig = px.scatter(reduced_table, x="datapoints", y="tokens_recovered", color="dataset", # symbol="model",
                                     labels={ # replaces default labels by column name
                    "experiment": "Experiment",  
                    "mean": f"Total Accuracy", 
                    "real_size": "Real Dataset Samples",
                    "datapoints": "Batch Size",
                    "dataset":"Dataset",
                    "tokens_recovered":"Total number of tokens leaked exactly"
                })
    fig.update_traces(mode='lines+markers', line=dict(width=5, dash='solid'))


    default_styling(fig)
    fig.update_xaxes(type="log")
    fig.update_yaxes(type="log")
    fig.update_layout(legend=dict(y=1, x=0.257))
    fig.show()

    if WRITE_TO_FILE:
        fig.write_image(f"images/decepticon_total_tokens_{model}_{seq_length}.pdf")
        fig.write_image(f"images/decepticon_total_tokens_{model}_{seq_length}.pdf")

# Token frequencies